# Expr

This notebook was automatically generated from the Algorithms for Optimization source code. Each cell generates a figure from the original text. While this code is not optimized for use in lectures, we provide it here to be adapted for such projects. We hope you find it useful.

In [ ]:
include("support_code.jl");

In [ ]:
    abstract type SelectionMethod end
    abstract type CrossoverMethod end
    abstract type MutationMethod end

    struct TruncationSelection <: SelectionMethod
        k::Int
    end

In [ ]:
    struct RouletteWheelSelection <: SelectionMethod end